# How to build models with limited amount of data

In [1]:
# lets import VGG16 pre-trained model
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',include_top=False,input_shape=(150, 150, 3))

In [2]:
conv_base.summary()

#### 3.3.2) FAST FEATURE EXTRACTION WITHOUT DATA AUGMENTATION

We’ll start by running instances of the previously introduced ImageDataGenerator to extract images as Numpy arrays as well as their labels. We’ll extract features from these images by calling the predict method of the conv_base model.

In [3]:
#base_dir = '/Users/fchollet/Downloads/cats_and_dogs_small'
#train_dir = os.path.join(base_dir, 'train')
#validation_dir = os.path.join(base_dir, 'validation')
#test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(directory,target_size=(150, 150),batch_size=batch_size,class_mode='binary')
    
    i=0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

In [4]:
train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)

In [5]:
train_features = np.reshape(train_features, (2000, 4*4* 512))
validation_features = np.reshape(validation_features, (1000, 4*4* 512))

At this point, we can define our densely connected classifier (note the use of dropout for regularization) and train it on the data and labels that we just recorded.

In [6]:
# Densly Connected Classifier
model = keras.models.Sequential()
model.add(keras.layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [7]:
# compile the model
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
loss='binary_crossentropy',
metrics=['acc'])

In [8]:
# fit the model on the features extracted
history = model.fit(train_features, train_labels,epochs=EPOCH_30,batch_size=20,validation_data=(validation_features, validation_labels))

Training is very fast, because we only have to deal with two Dense layers

In [9]:
# plotting the results
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

#### 3.3.2) FEATURE EXTRACTION WITH DATA AUGMENTATION

Now, let’s review the second technique we mentioned for doing feature extraction,which is much slower and more expensive, but which allows us to use data augmentation during training: extending the conv_base model and running it end to end on the inputs.

Because models behave just like layers, you can add a model (like conv_base) to a Sequential model just like you would add a layer.

In [10]:
#  Adding a densely connected classifier on top of the convolutional base
model = keras.models.Sequential()
model.add(conv_base) # adding pre-trained conv base model
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [11]:
model.summary()

In [ ]:
 print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

In [ ]:
# freezing the layers of pre-trained model
conv_base.trainable = False

In [ ]:
print('This is the number of trainable weights after freezing the conv base:', len(model.trainable_weights))

In [ ]:
#  Training the model end to end with a frozen convolutional base
train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    target_size=(150, 150),
                    batch_size=20,
                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_dir,target_size=(150, 150),batch_size=20,class_mode='binary')

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=2e-5),metrics=['acc'])

In [ ]:
history = model.fit_generator(
            train_generator,
            steps_per_epoch=100,
            epochs=EPOCH_30,
            validation_data=validation_generator,
            validation_steps=50)

Let’s plot the results again. 

Great, we reached a validation accuracy of about 90%. This is much better than we achieved with the small convnet trained from scratch.

In [ ]:
# plotting the results
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
conv_base.trainable = True
set_trainable = False

for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

Now we can begin fine-tuning the network. 

We’ll do this with the RMSProp optimizer, using a very low learning rate. The reason for using a low learning rate is that
we want to limit the magnitude of the modifications we make to the representations of the three layers we’re fine-tuning. 

Updates that are too large may harm these representations.

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=1e-5),metrics=['acc'])

history = model.fit_generator(train_generator,steps_per_epoch=100,epochs=EPOCH_100,validation_data=validation_generator,validation_steps=50)

In [ ]:
# plotting the results
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

We have 93% accuracy now, which is way better than the model that we build from scratch.
